In [1]:
import os, sys
import importlib
import torch

import numpy as np

from utils import load_torch, load

In [2]:
importlib.reload(load_torch)

data_folder = "data/simplified/"
access_file_generator = map(lambda x : data_folder + x + '.csv', load.classes)
trainGenerator = load_torch.ImageLoader(load.classes_1, data_folder)

In [3]:
gen = trainGenerator.__iter__()

In [4]:
%matplotlib inline
X, y = next(gen)
print('X.shape:', X.shape)
print('y.shape:', y.shape)

X.shape: torch.Size([16, 1, 32, 32])
y.shape: torch.Size([16])


# Building PyTorch CNN Model

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as ag
import torch.optim as optim


class VanillaDoodle(nn.Module):
    # Here we define our network structure
    name="doodle-vanilla"
    def __init__(self, cdim=8):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3).double() 
        self.conv2 = nn.Conv2d(6,16, 3).double()
        self.conv3 = nn.Conv2d(16,32,3).double()
        self.conv4 = nn.Conv2d(32,64,3).double()
        self.fc1   = nn.Linear(128,120).double()
        self.fc2   = nn.Linear(120, 84).double() 
        self.fc3   = nn.Linear(84,cdim).double()
        
    # Here we define one forward pass through the network
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        x = x.view(-1, int(self.num_flat_features(x)))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # Determine the number of features in a batch of tensors
    def num_flat_features(self, x): 
        size = x.size()
        return np.prod(size[1:])


#net = VanillaDoodle(cdim=len(load.classes_1))

In [6]:
# Code derived from: https://github.com/pytorch/vision/blob/master/torchvision/models/inception.py

class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class InceptionModule(nn.Module):

    def __init__(self, in_channels):
        super(InceptionModule, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 12, kernel_size=1)

        self.branch3x3_1 = BasicConv2d(in_channels, 24, kernel_size=1)
        self.branch3x3_2a = BasicConv2d(24, 24, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3_2b = BasicConv2d(24, 24, kernel_size=(3, 1), padding=(1, 0))

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 32, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(32, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3a = BasicConv2d(24, 24, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3dbl_3b = BasicConv2d(24, 24, kernel_size=(3, 1), padding=(1, 0))

        self.branch_pool = BasicConv2d(in_channels, 16, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3),
        ]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = [
            self.branch3x3dbl_3a(branch3x3dbl),
            self.branch3x3dbl_3b(branch3x3dbl),
        ]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)
    
class DoodleInception(nn.Module):
    # Here we define our network structure
    name="doodle-inception"
    def __init__(self, cdim=8):
        super(DoodleInception, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3).double() 
        self.mod1  = InceptionModule(6).double()
        self.conv2 = nn.Conv2d(124, 128, 3).double()
        self.fc1   = nn.Linear(512,128).double()
        self.fc2   = nn.Linear(128,cdim).double()
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.mod1(x)), (2, 2))  #?x124x7x7
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))  #?x128x2x2
        x = x.view(-1, int(self.num_flat_features(x)))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    # Determine the number of features in a batch of tensors
    def num_flat_features(self, x): 
        size = x.size()
        return np.prod(size[1:])

In [10]:
B     = 128              # Minibatch size
T     = 10               # Number of epochs
gamma = .001             # learning rate
rho   = .9               # momentum

classes = load.classes_1
    
net = DoodleInception(cdim=len(classes))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    net.parameters(),
    lr=gamma,
    momentum=rho
)

In [ ]:
if torch.cuda.is_available():
    net = net.cuda()

trainingAcc = [0]
testingAcc  = [0]
trainingLoss = []
testingLoss = []
for epoch in range(T):            
    gen = load_torch.ImageLoader(classes=load.classes_1,root_location=data_folder,read_size=32,batch_size=150)
    for i, (images, labels) in enumerate(gen):

        vsplit = int(images.shape[0]*0.8)
        
        xtrain = images[:vsplit]
        ltrain = labels[:vsplit]
        
        xtest  = images[vsplit:]
        ltest  = labels[vsplit:]
        
        # Forward pass
        ypred = net(xtrain)
        loss = criterion(ypred.cpu(), ltrain)
        train_acc  = 100 * np.mean(ltrain.data.numpy() == ypred.cpu().data.numpy ().T.argmax(axis =0))
        train_loss = loss.item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ypred = net(xtest)
        loss = criterion(ypred.cpu(), ltest)
        test_acc  = 100 * np.mean(ltest.data.numpy() == ypred.cpu().data.numpy ().T.argmax(axis =0))
        test_loss = loss.item()
        
        del images
        del labels
        del ypred

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}], Loss: {:.4f} (val:{:.4f}), Accuracy:{:.2f}% (val:{:.2f}%)' .format(epoch+1, T, i+1, train_loss, test_loss, train_acc, test_acc))
            #model.save_state_dict('mytraining.pt')
        if (i+1)%100==0:
            trainingAcc.append((0.1*train_acc) + (0.9*trainingAcc[-1]))
            trainingLoss.append(train_loss)
            testingAcc.append((0.1*test_acc) + (0.9*testingAcc[-1]))
            testingLoss.append(test_loss)

Epoch [1/10], Step [100], Loss: 1.9099 (val:1.8602), Accuracy:42.86% (val:31.82%)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(accuracy)
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.title("model accuracy")
plt.savefig("Graphs/{0}-{1}classes-acc.png".format(net.name, len(classes)))
plt.show()
plt.close()

plt.plot(loss_stored)
plt.ylabel("loss")
plt.xlabel("epochs")
plt.title("model loss")
plt.savefig("Graphs/{0}-{1}classes-loss.png".format(net.name, len(classes)))
plt.show()
plt.close()